# Import repo

In [1]:
!git clone https://github.com/FernandoRocha88/convert-streamlit-to-django.git

Cloning into 'convert-streamlit-to-django'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (408/408), done.
remote: Total 547 (delta 264), reused 349 (delta 107), pack-reused 0
Receiving objects: 100% (547/547), 406.43 KiB | 2.86 MiB/s, done.
Resolving deltas: 100% (264/264), done.


In [3]:
%cd /content/convert-streamlit-to-django/streamlit

/content/convert-streamlit-to-django/streamlit


In [4]:
pwd

'/content/convert-streamlit-to-django/streamlit'

# Load dataset

In [ ]:
! pip install streamlit

In [5]:
from src.processing.data_management import LoadIrisDataset

In [6]:
df = LoadIrisDataset()
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


* Train Test split

In [ ]:
from config import config
from src.processing.data_management import TrainTestSplit

In [ ]:
 X_train, X_test,y_train, y_test = TrainTestSplit(df=df,TARGET=config.ClfIrisSpecies_TARGET)

# Grid Search on 1 pipeline

In [ ]:
from config import config
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV

ClfIrisSpecies_DT = Pipeline(
    [       
        ("feat_selection",SelectFromModel(DecisionTreeClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", DecisionTreeClassifier(random_state=config.RANDOM_STATE))
    ]
)


_parameters = {
    'model__splitter': ["best","random"],
    'model__max_depth': [None,3,5,10],
    'model__criterion': ["gini", "entropy"]
}


_pipe = GridSearchCV(
		estimator = ClfIrisSpecies_DT,
		param_grid = _parameters, 
		cv=5,n_jobs=-2,verbose=2)

In [ ]:
_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__

[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, mo

[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:    0.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feat_selection',
                                        SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                         class_weight=None,
                                                                                         criterion='gini',
                                                                                         max_depth=None,
                                                                                         max_features=None,
                                                                                         max_leaf_nodes=None,
                                                                                         min_impurity_decrease=0.0,
                                                                                         min_impurity_s

In [ ]:
PipelineToDeploy = _pipe.best_estimator_
PipelineToDeploy

Pipeline(memory=None,
         steps=[('feat_selection',
                 SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                  class_weight=None,
                                                                  criterion='gini',
                                                                  max_depth=None,
                                                                  max_features=None,
                                                                  max_leaf_nodes=None,
                                                                  min_impurity_decrease=0.0,
                                                                  min_impurity_split=None,
                                                                  min_samples_leaf=1,
                                                                  min_samples_split=2,
                                                                  min_weight_fract

In [ ]:
_pipe.best_params_

{'model__criterion': 'gini',
 'model__max_depth': None,
 'model__splitter': 'best'}

In [ ]:
# main features in this pipeline -taken from feat selection step
X_train.columns[PipelineToDeploy['feat_selection'].get_support()].to_list()

['petal width (cm)']

In [ ]:
from sklearn.metrics import classification_report
print( classification_report(y_test, PipelineToDeploy.predict(X_test)) )

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.83      1.00      0.91        10
           2       1.00      0.80      0.89        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30

